In [1]:
import pickle
import pandas as pd
import re
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Load the model and encoder from the pickle file
with open('RF_algorithm_general.pkl', 'rb') as file:
    rf_classifier, onehot_encoder_features = pickle.load(file)

In [3]:
# # Load the model and encoder from the pickle file
# with open('RF_algorithm_1.0.1.pkl', 'rb') as file:
#     rf_classifier, onehot_encoder_features = pickle.load(file)

In [4]:
# # Load the model and encoder from the pickle file
# with open('RF_algorithm_1.0.2.pkl', 'rb') as file:
#     rf_classifier, onehot_encoder_features = pickle.load(file)

In [5]:
def extract_containers(content):
    container_pattern = re.compile(
        r'(Container|ContainerDb|System_Ext|Person)\((\w+), "(.*?)", "(.*?)", "(.*?)\\nSourceType: (.*?)"\)',
        re.DOTALL
    )
    containers = {}
    matches = container_pattern.findall(content)
    for match in matches:
        container_type, var_name, name, description, _, source_type = match
        containers[var_name.strip()] = source_type.strip()
    return containers

In [6]:
def parse_puml(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    
    # Extract containers to map their types
    containers = extract_containers(content)
    
    # Adjusted regex pattern to capture all relevant fields
    pattern = re.compile(
        r'Rel\((\w+)\s*,\s*(\w+)\s*,\s*"(.*?)"\s*,\s*"(.*?)"\)',
        re.DOTALL
    )
    
    matches = pattern.findall(content)
    data = []
    for match in matches:
        source, target, relationship, details = match
        source, target = source.strip(), target.strip()
        
        details_dict = {
            'SourceType': containers.get(source),
            'TargetType': containers.get(target),
            'AuthRequired': re.search(r'AuthRequired:\s*(.*?)(,|$)', details).group(1) if re.search(r'AuthRequired:\s*(.*?)(,|$)', details) else None,
            'Encryption': re.search(r'Encryption:\s*(.*?)(,|$)', details).group(1) if re.search(r'Encryption:\s*(.*?)(,|$)', details) else None,
            'EncryptionType': re.search(r'EncryptionType:\s*(.*?)(,|$)', details).group(1) if re.search(r'EncryptionType:\s*(.*?)(,|$)', details) else None,
            'DataFormat': re.search(r'DataFormat:\s*(.*?)(,|$)', details).group(1) if re.search(r'DataFormat:\s*(.*?)(,|$)', details) else None,
            'Frequency': re.search(r'Frequency:\s*(.*?)(,|$)', details).group(1) if re.search(r'Frequency:\s*(.*?)(,|$)', details) else None,
            'DataIntegrity': re.search(r'DataIntegrity:\s*(.*?)(,|$)', details).group(1) if re.search(r'DataIntegrity:\s*(.*?)(,|$)', details) else None,
            'AccessType': re.search(r'AccessType:\s*(.*?)(,|$)', details).group(1) if re.search(r'AccessType:\s*(.*?)(,|$)', details) else None,
            'AccessTarget': re.search(r'AccessTarget:\s*(.*?)(,|$)', details).group(1) if re.search(r'AccessTarget:\s*(.*?)(,|$)', details) else None,
            'NetworkProtocol': re.search(r'NetworkProtocol:\s*(.*?)(,|$)', details).group(1) if re.search(r'NetworkProtocol:\s*(.*?)(,|$)', details) else None,
            'CommunicationChannel': re.search(r'CommunicationChannel:\s*(.*?)(,|$)', details).group(1) if re.search(r'CommunicationChannel:\s*(.*?)(,|$)', details) else None,
            'CredentialStorage': re.search(r'CredentialStorage:\s*(.*?)(,|$)', details).group(1) if re.search(r'CredentialStorage:\s*(.*?)(,|$)', details) else None,
            'Interactor': re.search(r'Interactor:\s*(.*?)(,|$)', details).group(1) if re.search(r'Interactor:\s*(.*?)(,|$)', details) else None,
            'Threat': re.search(r'Threat:\s*(.*?)(,|$)', details).group(1) if re.search(r'Threat:\s*(.*?)(,|$)', details) else None,
        }
        
        data.append(details_dict)
    
    df = pd.DataFrame(data)
    return df

In [7]:
# Load and parse the PUML file
df = parse_puml('testpuml2.puml')

In [8]:
# List of columns used during training (excluding 'Threat')
training_columns = [
    'SourceType', 'TargetType', 'AuthRequired', 'Encryption', 'EncryptionType', 
    'DataFormat', 'Frequency', 'DataIntegrity', 'AccessType', 'AccessTarget', 
    'NetworkProtocol', 'CommunicationChannel', 'CredentialStorage', 'Interactor'
]

# Ensure columns are in the same order as during training and exclude unwanted columns
X_new = df[training_columns]

# Preprocess the data using the loaded encoder
X_new_encoded = onehot_encoder_features.transform(X_new)

# Convert to DataFrame to align feature names
df_columns = onehot_encoder_features.get_feature_names_out()
X_new_encoded_df = pd.DataFrame(X_new_encoded.toarray(), columns=df_columns)

# Make predictions
new_predictions = rf_classifier.predict(X_new_encoded_df)

# Add the predictions to the new data
df['Predicted_Threat'] = new_predictions

# Save the updated data to a new Excel file
#df.to_excel('new_data_with_predictions.xlsx', index=False)

# If true labels are available in the new data, calculate and display metrics
if 'Threat' in df.columns:
    y_true = df['Threat']
    y_pred = new_predictions
    
    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    report = classification_report(y_true, y_pred)
    
    print("\nAccuracy on new data:", accuracy)
    print("\nClassification Report on new data:\n", report)


Accuracy on new data: 0.5

Classification Report on new data:
                                       precision    recall  f1-score   support

   Cross-Site Request Forgery (CSRF)       0.33      1.00      0.50         1
                       Cryptojacking       0.00      0.00      0.00         1
                 Directory Traversal       0.00      0.00      0.00         0
Distributed Denial of Service (DDoS)       1.00      1.00      1.00         1
           Drive-by Download Attacks       1.00      1.00      1.00         1
     Man-in-the-Middle (MitM) Attack       0.00      0.00      0.00         1
                           No Threat       0.33      1.00      0.50         1
                    Phishing Attacks       0.00      0.00      0.00         1
                   Ransomware Attack       0.00      0.00      0.00         1
         Remote Code Execution (RCE)       1.00      1.00      1.00         1
                    Zero-day Exploit       0.00      0.00      0.00         1

C:\Users\Dennis\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Dennis\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Dennis\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Dennis\anaconda3\Lib\site-packages\s

In [9]:
df

,SourceType,TargetType,AuthRequired,Encryption,EncryptionType,DataFormat,Frequency,DataIntegrity,AccessType,AccessTarget,NetworkProtocol,CommunicationChannel,CredentialStorage,Interactor,Threat,Predicted_Threat
0,Device,Web Service/API,No,No,No Encryption,Binary,Event-Driven,No Data Integrity,Read,In-Memory Cache,No Network Protocol,No Communication Channel,No Credential Storage,Service,No Threat,No Threat
1,Software/Program,Database,No,Yes,Symmetric,XML,Batch,Hash,Write,Database,TCP/IP,Wired,No Credential Storage,System,Ransomware Attack,No Threat
2,Device,None,No,Yes,TLS/SSL,JSON,Real-Time,Checksum,Write,Web Service/API,MQTT,Wireless,No Credential Storage,Device,Man-in-the-Middle (MitM) Attack,Cross-Site Request Forgery (CSRF)
3,User,None,Yes,No,No Encryption,HTML,Real-Time,No Data Integrity,Write,User Interface,HTTP/HTTPS,Wireless,Plain Text,User,Cross-Site Request Forgery (CSRF),Cross-Site Request Forgery (CSRF)
4,Database,Service,Yes,Yes,Hashing,CSV,Batch,Digital Signature,Read&Write,Web Service/API,HTTP/HTTPS,Virtual Private Network,Hashed,Third-Party API,Distributed Denial of Service (DDoS),Distributed Denial of Service (DDoS)
5,Service,Web Application,Yes,Yes,Asymmetric,XML,Real-Time,Hash,Read&Write,Web Service/API,HTTP/HTTPS,Wired,Secure Vault,Application,Phishing Attacks,Cross-Site Request Forgery (CSRF)
6,Database,None,Yes,Yes,Hashing,CSV,Real-Time,Checksum,Write,Web Service/API,HTTP/HTTPS,Virtual Private Network,Hashed,Service,Remote Code Execution (RCE),Remote Code Execution (RCE)
7,Web Application,None,Yes,Yes,Hashing,JSON,Event-Driven,Digital Signature,Write,Web Service/API,HTTP/HTTPS,Wired,Hashed,Service,Drive-by Download Attacks,Drive-by Download Attacks
8,Database,System/Server,Yes,Yes,Asymmetric,XML,Real-Time,Hash,Read,Web Service/API,HTTP/HTTPS,Wired,Encrypted,User,Cryptojacking,No Threat
9,System/Server,Database,Yes,No,No Encryption,XML,On-Demand,No Data Integrity,Write,Web Service/API,HTTP/HTTPS,Virtual Private Network,Plain Text,User,Zero-day Exploit,Directory Traversal


In [10]:
# After making predictions and adding the predictions to the new data
df['Predicted_Threat'] = new_predictions

# Save the updated data to a new Excel file
df.to_excel('new_data_with_predictions.xlsx', index=False)

# Print the DataFrame to see the true labels and the predicted labels for each row
print(df[['Threat', 'Predicted_Threat']])


                                 Threat                      Predicted_Threat
0                             No Threat                             No Threat
1                     Ransomware Attack                             No Threat
2       Man-in-the-Middle (MitM) Attack     Cross-Site Request Forgery (CSRF)
3     Cross-Site Request Forgery (CSRF)     Cross-Site Request Forgery (CSRF)
4  Distributed Denial of Service (DDoS)  Distributed Denial of Service (DDoS)
5                      Phishing Attacks     Cross-Site Request Forgery (CSRF)
6           Remote Code Execution (RCE)           Remote Code Execution (RCE)
7             Drive-by Download Attacks             Drive-by Download Attacks
8                         Cryptojacking                             No Threat
9                      Zero-day Exploit                   Directory Traversal
